# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [68]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [69]:
filepath = "output_data/cities.csv"
citiesDF = pd.read_csv(filepath).drop('Unnamed: 0', axis=1) 
citiesDF.head()

,City,Country,datetime,lat,long,Max Temp (F),Cloudiness,Humidity,Wind Speed
0,Ushuaia,AR,1644551152,-54.8000,-68.3000,44.3,40,81,12.86
1,Doha,QA,1644551347,25.2867,51.5333,64.3,0,85,1.54
2,Jamestown,US,1644551347,42.0970,-79.2353,32.1,100,59,3.09
3,Huittinen,FI,1644551348,61.1833,22.7000,28.0,11,92,3.16
4,Gawler,AU,1644551138,-34.6000,138.7333,83.1,28,34,3.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [82]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure(zoom_level=2, center=[10, 0])
heat_layer = gmaps.heatmap_layer(citiesDF[["lat","long"]], weights=citiesDF['Humidity'], max_intensity=citiesDF['Humidity'].max())
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [73]:
# narrow down the cities...
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
conditions_df = (
    (citiesDF['Max Temp (F)'] < 80) 
    & (citiesDF['Max Temp (F)'] > 70) 
    & (citiesDF['Wind Speed'] < 10)
    & (citiesDF['Cloudiness'] == 0)
    & (citiesDF['Humidity'] < 80) # I added this to get the number of cities down to 10
                )
hotel_df = citiesDF.loc[conditions_df]
# how many rows are there? don't want to make too many api calls lol
print(hotel_df.shape[0])

10


In [74]:
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,datetime,lat,long,Max Temp (F),Cloudiness,Humidity,Wind Speed
0,Hilo,US,1644551364,19.7297,-155.0900,74.9,0,77,2.57
1,Pisco,PE,1644551087,-13.7000,-76.2167,71.7,0,73,4.12
2,Coronado,US,1644551139,32.6859,-117.1831,71.4,0,39,4.63
3,Orange,US,1644551416,33.7000,-117.7675,76.1,0,31,1.35
4,Burnie,AU,1644551490,-41.0667,145.9167,76.9,0,33,4.29
5,Cockburn Town,TC,1644551235,21.4612,-71.1419,78.0,0,78,8.26
6,Mercedes,AR,1644551503,-34.6515,-59.4307,70.5,0,73,3.54
7,Bonito,BR,1644551509,-21.1211,-56.4819,72.5,0,76,2.05
8,Port Hueneme,US,1644551521,34.1478,-119.1951,76.2,0,47,2.57
9,Pozo Colorado,PY,1644551522,-23.4923,-58.7972,74.9,0,34,1.90


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [76]:
request_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [77]:
for index, row in hotel_df.iterrows():
    lat = row['lat']
    long = row['long']
    params['location'] = f'{lat}, {long}'
    try:
        response = requests.get(request_url, params=params).json()
        hotel_name = response['results'][0]['name']
        hotel_df.at[index, 'Hotel Name'] = hotel_name
    except:
        pass
hotel_df

/Users/randyweaver/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/randyweaver/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Country,datetime,lat,long,Max Temp (F),Cloudiness,Humidity,Wind Speed,Hotel Name
0,Hilo,US,1644551364,19.7297,-155.0900,74.9,0,77,2.57,Hilo Hawaiian Hotel
1,Pisco,PE,1644551087,-13.7000,-76.2167,71.7,0,73,4.12,Hotel San Isidro Oficial
2,Coronado,US,1644551139,32.6859,-117.1831,71.4,0,39,4.63,El Cordova Hotel
3,Orange,US,1644551416,33.7000,-117.7675,76.1,0,31,1.35,Meadows
4,Burnie,AU,1644551490,-41.0667,145.9167,76.9,0,33,4.29,Beachfront Voyager Motor Inn
5,Cockburn Town,TC,1644551235,21.4612,-71.1419,78.0,0,78,8.26,Osprey Beach Hotel
6,Mercedes,AR,1644551503,-34.6515,-59.4307,70.5,0,73,3.54,Posada del Inti Centro
7,Bonito,BR,1644551509,-21.1211,-56.4819,72.5,0,76,2.05,Marruá Hotel
8,Port Hueneme,US,1644551521,34.1478,-119.1951,76.2,0,47,2.57,Hampton Inn Channel Islands Harbor
9,Pozo Colorado,PY,1644551522,-23.4923,-58.7972,74.9,0,34,1.90,Hospedaje la Amistad


In [84]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [85]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))